In [1]:
import os
import shutil
from pathlib import Path

In [2]:
INPUT_FOLDER = "../datasets/LongSpeechSource/TEDLIUM_release-3"
OUTPUT_FOLDER = "./tedlium"

In [3]:
def collect_subset(
 tedlium_root: Path,
 output_dir: Path,
 num_files: int = 1
):
    # 原始路径
    sph_dir = Path(os.path.join(tedlium_root, "data", "sph"))
    stm_dir = Path(os.path.join(tedlium_root, "data", "stm"))
    out_sph_dir = Path(os.path.join(output_dir, "data", "sph"))
    out_stm_dir = Path(os.path.join(output_dir, "data", "stm"))
    out_sph_dir.mkdir(parents=True, exist_ok=True)
    out_stm_dir.mkdir(parents=True, exist_ok=True)

    # 找到所有 .sph 文件
    sph_files = sorted(sph_dir.glob("*.sph"))
    selected = sph_files[:num_files]
    print(sph_dir)
    for sph_path in selected:
        base = sph_path.stem  # TED_2008_0001
        stm_path = stm_dir / f"{base}.stm"

        if not stm_path.exists():
            print(f"⚠️ STM not found for: {base}")
            continue

        # 复制 .sph 和对应 .stm
        shutil.copy(sph_path, out_sph_dir / sph_path.name)
        shutil.copy(stm_path, out_stm_dir / stm_path.name)

        print(f" Copied: {base}.sph and {base}.stm")

In [4]:
collect_subset(INPUT_FOLDER, OUTPUT_FOLDER)

../datasets/LongSpeechSource/TEDLIUM_release-3/data/sph
 Copied: 911Mothers_2010W.sph and 911Mothers_2010W.stm
